In [ ]:
# Import libraries
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
import numpy as np
import pandas_datareader as pdr
plt.style.use('fivethirtyeight')
import warnings
warnings.filterwarnings('ignore')
import tqdm
import pandas as pd
import random
from fbprophet import Prophet

In [ ]:
# Function to get pricing
def grab_dat(ticker):
    
    dat = pdr.get_data_yahoo(ticker, '2000-01-01', pd.to_datetime('today', format = '%Y-%m-%d')).rename({'Adj Close': 'price'}, axis = 1)[['price']]
        
    dat['ticker'] = ticker

    # Returns
    dat['daily_pct_change'] = dat['price'] / dat['price'].shift(1) - 1
    dat['daily_pct_change'].fillna(0, inplace = True)
    dat['cum_daily_return'] = (1 + dat['daily_pct_change']).cumprod()
        
    return dat

In [ ]:
# Get tickers
table = pd.read_html('https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')
tickers = table[0]

tickers.groupby(['GICS Sector', 'GICS Sub-Industry']).size()
tickers['GICS Sector'].value_counts()

tickers1 = {}

for nm, grp in tickers.groupby('GICS Sector'):
    
    tickers1[nm] = [i for i in grp['Symbol'].unique()]

In [ ]:
ticker = random.choice(tickers['Symbol'].unique())
print(f'Stock selected for evaluation: {ticker}')

In [ ]:
df = grab_dat(ticker)

In [ ]:
df.tail()

In [ ]:
df['short'] = df['price'].rolling(30).mean()
df['long'] = df['price'].rolling(90).mean()

In [ ]:
# Separate data
training_data = df['2018':]
validation_data = df[:'2018']

In [ ]:
fig, ax = plt.subplots(figsize = (25, 10))
ax.plot(training_data['price'], label = 'training', lw = 1)
ax.plot(validation_data['price'], label = 'vailation', lw = 1)
ax.plot(df['short'], label = '30-day SMA', lw = .5)
ax.plot(df['long'], label = '90-day SMA', lw = .5)
plt.title(f'Price and Data Split for {ticker}', fontsize = 12)
plt.xlabel('Date', fontsize = 12)
plt.ylabel('Price ($)', fontsize = 12)
fig.legend(fontsize = 12)
fig.tight_layout()

In [ ]:
# Backtest actual data
validation_data['signal'] = 0
validation_data['signal'][30:] = np.where(validation_data['short'][30:] > validation_data['long'][30:], 1.0, 0.0)   
validation_data['positions'] = validation_data['signal'].diff()

positions = pd.DataFrame(index = validation_data.index).fillna(0.0)
positions[nm] = 1000 * validation_data['signal'] 

port = positions.multiply(validation_data['price'], axis = 0)
pos_diff = positions.diff()

# Calculate totals
port['holdings'] = (positions.multiply(validation_data['price'], axis = 0)).sum(axis = 1)
port['cash'] = 10000 - (pos_diff.multiply(validation_data['price'], axis = 0)).sum(axis = 1).cumsum()   
port['total'] = port['cash'] + port['holdings']
port['returns'] = port['total'].pct_change() 

In [ ]:
# Initialize model
mod = Prophet()
mod.fit(validation_data.reset_index().rename({'Date': 'ds', 'price': 'y'}, axis = 1))

fut = mod.make_future_dataframe(periods = 36)
fcst = mod.predict(fut)

In [ ]:
# Backtest actual data
validation_data['signal'] = 0
validation_data['signal'][30:] = np.where(validation_data['short'][30:] > validation_data['long'][30:], 1.0, 0.0)   
validation_data['positions'] = validation_data['signal'].diff()

positions = pd.DataFrame(index = validation_data.index).fillna(0.0)
positions[ticker] = 1000 * validation_data['signal'] 

port = positions.multiply(validation_data['price'], axis = 0)
pos_diff = positions.diff()

# Calculate totals
port['holdings'] = (positions.multiply(validation_data['price'], axis = 0)).sum(axis = 1)
port['cash'] = 10000 - (pos_diff.multiply(validation_data['price'], axis = 0)).sum(axis = 1).cumsum()   
port['total'] = port['cash'] + port['holdings']
port['returns'] = port['total'].pct_change() 

In [ ]:
port.tail()

In [ ]:
# Backtest predicted data
fcst['short'] = fcst['yhat'].rolling(30).mean()
fcst['long'] = fcst['yhat'].rolling(90).mean()

fcst.index = pd.DatetimeIndex(fcst['ds'])

# Backtest actual data
fcst['signal'] = 0
fcst['signal'][30:] = np.where(fcst['short'][30:] > fcst['long'][30:], 1.0, 0.0)   
fcst['positions'] = fcst['signal'].diff()

pred_positions = pd.DataFrame(index = validation_data.index).fillna(0.0)
pred_positions[ticker] = 1000 * fcst['signal'] 

predport = pred_positions.multiply(validation_data['price'], axis = 0)
pred_pos_diff = positions.diff()

# Calculate totals
predport['holdings'] = (predport.multiply(validation_data['price'], axis = 0)).sum(axis = 1)
predport['cash'] = 10000 - (pred_pos_diff.multiply(validation_data['price'], axis = 0)).sum(axis = 1).cumsum()   
predport['total'] = predport['cash'] + predport['holdings']
predport['returns'] = predport['total'].pct_change() 

In [ ]:
predport.tail()

In [ ]:
fig, ax = plt.subplots(figsize = (25, 10))
ax.plot(port['total'], label = 'actual', lw = 1)
ax.plot(predport['total'], label = 'predicted', lw = 1)
plt.title(f'Performance 2018-2021 for {ticker}', fontsize = 12)
plt.xlabel('Date', fontsize = 12)
plt.ylabel('Total ($)', fontsize = 12)
fig.legend(fontsize = 12)
fig.tight_layout()